In [2]:
!pip install -q pandas pandas-gbq google-cloud-bigquery

from google.colab import auth
auth.authenticate_user()

import pandas as pd
pd.set_option('display.max_columns', None)

PROJECT_ID = "my-educational-project-476420"
DATASET = "thelook_ecommerce_project"

In [3]:
from pandas_gbq import read_gbq
import numpy as np

In [4]:
tables = [
    "aov_by_category",
    "basket_pairs_base",
    "category_revenue",
    "cohorts_base",
    "monthly_revenue",
    "new_vs_repeat",
    "pareto_products",
    "rfm_base"
]

dfs = {}
for t in tables:
    sql = f"SELECT * FROM `{PROJECT_ID}.{DATASET}.{t}`"
    dfs[t] = read_gbq(sql, project_id=PROJECT_ID)
    print(f"{t}: {dfs[t].shape}")

Downloading: 100%|██████████|
aov_by_category: (26, 2)
Downloading: 100%|██████████|
basket_pairs_base: (45194, 2)
Downloading: 100%|██████████|
category_revenue: (26, 5)
Downloading: 100%|██████████|
cohorts_base: (125048, 3)
Downloading: 100%|██████████|
monthly_revenue: (83, 2)
Downloading: 100%|██████████|
new_vs_repeat: (2, 3)
Downloading: 100%|██████████|
pareto_products: (9858, 4)
Downloading: 100%|██████████|
rfm_base: (27456, 4)


In [5]:
c = dfs['cohorts_base'].copy()
c["cohort_month"] = pd.to_datetime(c["cohort_month"]).dt.to_period("M").dt.to_timestamp()
c["order_month"]  = pd.to_datetime(c["order_month"]).dt.to_period("M").dt.to_timestamp()

c["month_to_order"] = ((c["order_month"].dt.year - c["cohort_month"].dt.year) * 12 +
                     (c["order_month"].dt.month - c["cohort_month"].dt.month)).astype(int)

In [6]:
cohort_counts = c.groupby(['cohort_month', 'month_to_order'])['user_id'].nunique().reset_index()
cohort_counts = cohort_counts.rename(columns={'user_id': 'customers_amount'})

first_purchases = cohort_counts[cohort_counts['month_to_order'] == 0][['cohort_month', 'customers_amount']].copy()
first_purchases = first_purchases.rename(columns={'customers_amount': 'first_purchases'})

all_months_to_order = cohort_counts['month_to_order'].unique()
all_cohort_months = cohort_counts['cohort_month'].unique()
from itertools import product
complete_grid = pd.DataFrame(list(product(all_cohort_months, all_months_to_order)), columns=['cohort_month', 'month_to_order'])

cohort_counts = pd.merge(complete_grid, cohort_counts[['cohort_month', 'month_to_order', 'customers_amount']], on=['cohort_month', 'month_to_order'], how='left')

cohort_counts = pd.merge(cohort_counts, first_purchases, on='cohort_month', how='left')

cohort_counts['customers_amount'] = cohort_counts['customers_amount'].fillna(0).astype(int)

cohort_counts['month_to_order'] = cohort_counts['month_to_order'].astype(int)

cohort_counts = cohort_counts.sort_values(by=['cohort_month', 'month_to_order']).reset_index(drop=True)


display(cohort_counts.head())

,cohort_month,month_to_order,customers_amount,first_purchases
0,2019-01-01,0,11,11
1,2019-01-01,1,0,11
2,2019-01-01,2,0,11
3,2019-01-01,3,0,11
4,2019-01-01,4,0,11


In [52]:
cohort_counts.columns = cohort_counts.columns.astype(str)

cohort_counts['cohort_month'] = pd.to_datetime(cohort_counts['cohort_month'])

destination_table = f"{PROJECT_ID}.{DATASET}.cohorts_looker"
to_gbq(cohort_counts, destination_table, project_id=PROJECT_ID, if_exists='replace')

print(f"cohort_counts uploaded to BigQuery table: {destination_table}")

100%|██████████| 1/1 [00:00<00:00, 7913.78it/s]

cohort_counts uploaded to BigQuery table: my-educational-project-476420.thelook_ecommerce_project.cohorts_looker


**RFM**

In [8]:
rfm = dfs['rfm_base'].copy()

r_labels = [5,4,3,2,1]
rfm["R"] = pd.qcut(rfm["recency_days"], 5, labels=r_labels).astype(int)

rfm["F"] = pd.qcut(rfm["frequency"].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
rfm["M"] = pd.qcut(rfm["monetary"].rank(method="first"),  5, labels=[1,2,3,4,5]).astype(int)

rfm["RFM_score"] = rfm["R"].astype(str)+rfm["F"].astype(str)+rfm["M"].astype(str)

def segment(row):
    if row["R"]>=4 and row["F"]>=4 and row["M"]>=4:
        return "Champions"
    if row["R"]>=4 and row["F"]>=3:
        return "Loyal"
    if row["R"]<=2 and row["F"]<=2 and row["M"]<=2:
        return "At Risk/Churn"
    return "Regular"

rfm["segment"] = rfm.apply(segment, axis=1)

rfm_csv_path = "/content/rfm_segments.csv"
rfm.to_csv(rfm_csv_path, index=False)

rfm_summary = (rfm.groupby("segment")
                 .agg(users=("user_id","nunique"),
                      avg_R=("R","mean"),
                      avg_F=("F","mean"),
                      avg_M=("M","mean"),
                      total_monetary=("monetary","sum"))
                 .sort_values("total_monetary", ascending=False))
rfm_summary.head(10)

destination_table = f"{PROJECT_ID}.{DATASET}.rfm"
to_gbq(rfm, destination_table, project_id=PROJECT_ID, if_exists='replace')

print(f"rfm DataFrame uploaded to BigQuery table: {destination_table}")

100%|██████████| 1/1 [00:00<00:00, 11244.78it/s]

rfm DataFrame uploaded to BigQuery table: my-educational-project-476420.thelook_ecommerce_project.rfm
